In [2]:
%pylab inline
%load_ext Cython

Populating the interactive namespace from numpy and matplotlib


SyntaxError: invalid syntax (<ipython-input-5-5db33f195660>, line 1)

In [3]:
NumRuns = 1
NumGenerations = 10
PopulationSize = 10
MutationRate = 0.01
ExecutionError= 0.001
ReputationAssignmentError = 0.001
PrivateAssessmentError = 0.001
ReputationUpdateProbability = 1.0
RandomSeed =  666
SocialNormMatrix = [[0, 0],[0, 0]]
CostValue = 1
BenefitValue = 5

In [6]:
%%timeit
sim.run_instance(NumRuns, NumGenerations, PopulationSize, MutationRate,
                 ExecutionError, ReputationAssignmentError,
                 PrivateAssessmentError, ReputationUpdateProbability,
                 RandomSeed, SocialNormMatrix, CostValue, BenefitValue)

10 loops, best of 3: 29.1 ms per loop


In [11]:
%%timeit
sim.run_instance(NumRuns, NumGenerations, PopulationSize, MutationRate,
                 ExecutionError, ReputationAssignmentError,
                 PrivateAssessmentError, ReputationUpdateProbability,
                 RandomSeed, SocialNormMatrix, CostValue, BenefitValue)

10 loops, best of 3: 28.1 ms per loop


In [12]:
%%timeit
for i in range(100):
    np.random.rand() < 0.001

10000 loops, best of 3: 22.3 µs per loop


In [14]:
%%timeit
ans = np.random.rand(100) < 0.001

The slowest run took 11.30 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 3.02 µs per loop


In [15]:
ans

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False, False], dtype=bool)

# Fast

In [76]:
my_list_fast = []
for _ in range(1000):
    population_size = 10
    population = [10, 0, 0, 2]
    index_focal, index_other = np.random.choice(range(len(population)), size = 2, replace=False)
    my_list_fast.append(tuple((index_focal, index_other)))

In [77]:
counter = Counter(my_list_fast)

In [78]:
counter

Counter({(0, 3): 839, (3, 0): 161})

# Dumb but real

In [79]:
%%timeit
my_list  = []
for i in range(1000):
    population_size = 10
    population = [10, 0, 0, 5]
    index_focal= np.random.choice(range(len(population)), 
                         p=population/np.sum(population))
    population[index_focal] -=1
    index_other= np.random.choice(range(len(population)), 
                 p=population/np.sum(population))
    my_list.append(tuple((index_focal, index_other)))    

10 loops, best of 3: 72.7 ms per loop


In [167]:
from collections import Counter

In [65]:
counter = Counter(my_list)

In [66]:
counter

Counter({(0, 0): 436, (0, 3): 232, (3, 0): 233, (3, 3): 99})

In [7]:
%%timeit
agent_pairs = [np.random.choice(500, size=2, replace=False)]*500

1000 loops, best of 3: 283 µs per loop


In [10]:
%%timeit
agent_pairs = np.array([np.random.choice(500, size=2, replace=False)]*500)

1000 loops, best of 3: 911 µs per loop


In [30]:
from collections import Counter

In [31]:
Z=5

In [6]:
%%timeit
for _ in range(10):
    for _ in range(3*np.power(10, 3)):
        agent_pairs = [np.random.choice(Z, size=2, replace=False)]*Z

1 loop, best of 3: 2.65 s per loop


In [75]:
%%timeit
for _ in range(300):
    my_list = [np.random.choice(Z, size=2, replace=False) for _ in range(1000)]

1 loop, best of 3: 10.3 s per loop


In [71]:
%%timeit
my_list = []
for _ in range(1000):
    my_list.append(np.random.choice(Z, size=2, replace=False))

10 loops, best of 3: 48.1 ms per loop


In [72]:
%%timeit
my_list = [None]*1000
for i in range(1000):
    my_list[i] = np.random.choice(Z, size=2, replace=False)

10 loops, best of 3: 48 ms per loop


In [53]:
counter = Counter(my_list)
print(counter)

Counter({(4, 2): 1000})


In [58]:
np.zeros((Z, 2))

array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.]])

In [76]:
%%timeit
for _ in range(10):
    for _ in range(3*np.power(10, 3)):
        agent_pairs = np.random.choice(Z, size=2, replace=False)

1 loop, best of 3: 1.48 s per loop


# Cython speedup tests

In [36]:
%reload_ext Cython
import pyximport
pyximport.install(setup_args={"include_dirs":np.get_include()},
                  reload_support=True)
import SimulationInstance as sim
import Cython_resources.simulation_instance as sim_c
%reload_ext Cython_resources.simulation_instance

In [42]:
runs = 1
generations = 100
population_size = 20
mutation_rate = 0.0
execution_error = 0.0
reputation_assignment_error = 0.0
private_assessment_error = 0.0
reputation_update_prob = 1.0
socialnorm = np.array([[1, 0], [0, 1]])
cost = 1
benefit = 5

### Pure python implementation

In [43]:
%%timeit
sim.simulate(runs, generations, population_size, mutation_rate,
             execution_error, reputation_assignment_error,
             private_assessment_error, reputation_update_prob,
             socialnorm, cost, benefit)

1 loop, best of 3: 4.39 s per loop


In [44]:
%%timeit
sim_c.run_instance(runs, generations, population_size, mutation_rate,
                execution_error, reputation_assignment_error,
                private_assessment_error, reputation_update_prob,
                socialnorm, cost, benefit)

1 loop, best of 3: 3.63 s per loop
